In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import logging
import importlib
import urllib
import unicodecsv as csv
from lxml import html
import requests
import pandas as pd
import matplotlib.pyplot as plt, numpy as np
import json
import sys
from requests.auth import HTTPProxyAuth
import time
import os

In [ ]:
#Script is written right now for before cleaned, single region. Must adapt for cleaned and multi-region.
#This will include changing "price" to "rent"
#QUESTION: Unclear what is actually being counted in 'score' variable

#cleandf = pd.read_csv('C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\github\\scraper2\\shared_room_scraper\\sfbay_merged_2017_04_10.csv', header=0) #just SF dataframe
cleandf = pd.read_csv('C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\github\\scraper2\\shared_room_scraper\\all_data_merged_2017_04_06.csv',error_bad_lines=False, header=0) #all listings dataframe
#determine rent per square foot


In [ ]:
cleandf.shape

In [ ]:
cleandf.loc[cleandf.lat == 99, 'lat'] = 0
cleandf.loc[cleandf.lng == 99, 'lng'] = 0
cleandf.loc[cleandf.sqft == 0, 'sqft'] = np.nan

In [ ]:
#For any duplicate post, we want to keep the version with the most information. Therefore, assign a score to each post and 
#give one point for square footage, lat, long and price
#OLD FORMULA
#cleandf['score']=cleandf.astype(bool).sum(axis=1)

cleandf['price_exists'] = cleandf['price']>0
cleandf['sqft_exists'] = cleandf['sqft']>0
cleandf['lat_exists'] = cleandf['lat']>0
cleandf['lng_exists'] = cleandf['lng']<0
cleandf['score'] = cleandf[['price_exists','sqft_exists','lat_exists','lng_exists']].astype(bool).sum(axis=1)

In [ ]:
#Sort rows by score
cleandf = cleandf.sort_values(by='score',ascending=False)

In [ ]:
cleandf.shape

In [ ]:
cleandf.tail()

In [ ]:
#List as 'true' if the PID, body_text, or the title are duplicates
cleandf['body_text_duplicate']=cleandf.duplicated('body_text')
cleandf['title_duplicate']=cleandf.duplicated('title')
cleandf['PID_duplicate']=cleandf.duplicated('pid')


In [ ]:
cleandf_unique = cleandf.drop_duplicates('pid')
cleandf_unique.shape

In [ ]:
#Keep rows with highest score. Drop rows with duplicate PIDs, Duplicate Titles, Duplicate Posting Bodies
cleandf_unique = cleandf_unique.drop_duplicates('body_text')
cleandf_unique.shape

In [ ]:
cleandf_unique = cleandf_unique.drop_duplicates('title')
cleandf_unique.shape

## Creating new fields for analysis

In [ ]:
cleandf_unique['dt'] = pd.to_datetime(cleandf_unique['dt'], format='%Y-%m-%d')
cleandf_unique['region'] = cleandf_unique['url'].str.extract('http://(.*).craigslist.org', expand=False)
cleandf_unique['day_of_week'] = cleandf_unique['dt'].apply(lambda x: x.weekday())
cleandf_unique['rent_sqft'] = cleandf_unique['price'] / cleandf['sqft']

In [ ]:
#counting listings per day (right now inlcudes time, need to remove)
listings_per_date = cleandf_unique['dt'].value_counts()

## Descriptive Stats and More Filtering

In [ ]:
cleandf_unique.describe()

In [ ]:
unique_wprice = cleandf_unique[cleandf_unique['price'] > 0]

In [ ]:
# in this cell, define the values by which we will filter the 3 columns
upper_percentile = 0.995
lower_percentile = 0.005

# how many rows would be within the upper and lower percentiles?
upper = int(len(unique_wprice) * upper_percentile)
lower = int(len(unique_wprice) * lower_percentile)

# get the rent/sqft values at the upper and lower percentiles
rent_sqft_sorted = unique_wprice['rent_sqft'].sort_values(ascending=True, inplace=False)
upper_rent_sqft = rent_sqft_sorted.iloc[upper]
lower_rent_sqft = rent_sqft_sorted.iloc[lower]

# get the rent values at the upper and lower percentiles
rent_sorted = unique_wprice['price'].sort_values(ascending=True, inplace=False)
upper_rent = rent_sorted.iloc[upper]
lower_rent = rent_sorted.iloc[lower]

# get the sqft values at the upper and lower percentiles
sqft_sorted = unique_wprice['sqft'].sort_values(ascending=True, inplace=False)
upper_sqft = sqft_sorted.iloc[upper]
lower_sqft = sqft_sorted.iloc[lower]

print('valid rent_sqft range:', [lower_rent_sqft, upper_rent_sqft])
print('valid rent range:', [lower_rent, upper_rent])
print('valid sqft range:', [lower_sqft, upper_sqft])

In [ ]:
#Filtering out rows with unreasonable rent prices 
rent_mask = (unique_wprice['price'] > lower_rent) & (unique_wprice['price'] < upper_rent)

In [ ]:
filtered_listings = pd.DataFrame(unique_wprice[rent_mask])

In [ ]:
filtered_listings.describe()

## Groupby Function for Viewing Results by Region

In [ ]:
grouped = cleandf_unique.groupby('region')

## Visualization

In [ ]:
x = filtered_listings['price']

In [ ]:
x = x.astype('int')

In [ ]:
plt.figure(1, figsize=(10,6), )
plt.suptitle('Rent Price Distribution')
plt.xlabel('Price')
plt.ylabel('Number of Listings')
ax = plt.hist(x, bins=50, alpha=.4, color='cyan')
plt.show()

In [ ]:
#filtered_listings.to_csv('sfbay_filtered4_10.csv')

In [ ]:
# calculate the ratios of unique to duplicate listings for each region
listings_ratios = pd.DataFrame()

# number of total listings for each region
listings_ratios['cleandf'] = cleandf['region'].value_counts()

# number of duplicate listings for each region (ie, listings that share a pid with at least one other listing)
listings_ratios['duplicate_listings'] = duplicate_listings['region'].value_counts()

# number of unique listings for the region (ie, none share a pid with another listing)
listings_ratios['unique_listings'] = unique_listings['region'].value_counts()

# percent of this region's listings that are duplicates
listings_ratios['duplicate_ratio'] = listings_ratios['duplicate_listings'] / listings_ratios['all_listings']

# percent of this region's listings that are unique (ie, not duplicates)
listings_ratios['unique_ratio'] = listings_ratios['unique_listings'] / listings_ratios['all_listings']

listings_ratios.head()